In [1]:
# Import necessary modules
import cvxpy as cvx
import numpy as np
from scipy import random
from scipy import linalg

In [2]:
def generate_points(num, dim):
    return np.matrix(4 * np.random.random((num, dim)) - 2)

In [3]:
def sum_elem_product(A,B):
    return cvx.sum_entries(cvx.mul_elemwise(A, B))

In [4]:
def generate_ss_distances(sensors):
    return list(map(lambda s1: 
                list(map(lambda s2: np.linalg.norm(s1 - s2), sensors))
            , sensors))

def generate_sa_distances(sensors, anchors):
    return list(map(lambda s: 
                list(map(lambda a: np.linalg.norm(a - s), anchors))
            , sensors))

In [5]:
def get_SOCP_result(sensors, anchors):
    d_ss = generate_ss_distances(sensors)
    d_sa = generate_sa_distances(sensors, anchors)
    
    x = cvx.Variable(num_sensors, dim)
    objective = cvx.Minimize(0)
    
    constraints  = []
    for i in range(num_sensors):
        x_i = x[i, :]
        for j in range(num_anchors):
            constraints.append(cvx.norm(x_i - anchors[j]) <= d_sa[i][j])
        for j in range(num_sensors):
            if i < j:
                constraints.append(cvx.norm(x_i - sensors[j]) <= d_ss[i][j])

    prob = cvx.Problem(objective, constraints)
    result = prob.solve(solver = 'MOSEK')
    print("SOCP complete")
    return x.value

In [6]:
def get_SDP_result(sensors, anchors):    
    
    A = enforce_id(sensors)
    B = enforce_id2(sensors)
    C = sensor_constraints(sensors)
    D = anchor_constraints(sensors, anchors)

    Z = cvx.Semidef(num_sensors + dim)
    
    constraints = []
    for id_constraint, rhs in zip(A[0], A[1]):
        constraints.append(sum_elem_product(id_constraint, Z) == rhs)
    for id_constraint2, rhs in zip(B[0], B[1]):
        constraints.append(sum_elem_product(id_constraint2, Z) == rhs)
    for sensor_constraint, rhs in zip(C[0], C[1]):
        constraints.append(sum_elem_product(sensor_constraint, Z) == rhs)
    for anchor_constraint, rhs in zip(D[0], D[1]):
        constraints.append(sum_elem_product(anchor_constraint, Z) == rhs)
        
    objective = cvx.Minimize(0)
    prob = cvx.Problem(objective, constraints)
    result = prob.solve(solver = 'MOSEK')

    print("SDP complete")
    return np.transpose(Z[0:dim, dim:dim+num_sensors].value)

#Make first set of constraint matrices (look like identity)
def enforce_id(sensors):
    dim = np.shape(sensors)[1]
    num_sensors = np.shape(sensors)[0]
    matrices = []
    rhs = []
    for i in range(dim):
        new_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
        new_matrix[i,i] = 1
        matrices.append(new_matrix)
        rhs.append(1)
        
    return (matrices, rhs)

#Make second set of constraint matrices (symmetric holders) 
def enforce_id2(sensors):
    dim = np.shape(sensors)[1]
    num_sensors = np.shape(sensors)[0]
    matrices = []
    rhs = []
    for i in range(dim):
        for j in range(dim):
            new_matrix = np.identity(dim)
            if(j > i):
                new_matrix[i,j] = 1
                new_matrix[j,i] = 1
                big_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
                big_matrix[0:dim,0:dim] = new_matrix
                matrices.append(big_matrix)
                rhs.append(dim)
                
    return (matrices, rhs)


#Make third set of constraint matrices (anchors to sensors)
def sensor_constraints(sensors):
    dim = np.shape(sensors)[1]
    num_sensors = np.shape(sensors)[0]
    d_ss = generate_ss_distances(sensors)
    matrices = []
    rhs = []
    zero_vec_dim = np.zeros(dim)
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[i] = 1
            zero_vec_num_s[j] = -1
            
            new_vec = np.matrix(np.append(zero_vec_dim, zero_vec_num_s))
            
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            
            matrices.append(new_matrix)
            rhs.append(d_ss[i][j]**2)

    return (matrices, rhs)

#Make fourth set of constraint matrices (sensors to sensors)
def anchor_constraints(sensors, anchors):
    num_anchors = np.shape(anchors)[0]
    d_sa = generate_sa_distances(sensors, anchors)
    matrices = []
    rhs = []
    for i in range(num_anchors):
        for j in range(num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[j] = -1

            new_vec = np.append(np.array(anchors[i,:]), np.array(zero_vec_num_s))
            new_vec = np.matrix(new_vec)
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            matrices.append(new_matrix)
            rhs.append(d_sa[j][i]**2)

    return (matrices, rhs)

In [7]:
def get_NLLS_matches(sensors, anchors):
    check = 1000
    max_iter = 10000
    k = 0

    # Initial sensors guess
    sensors_0 = generate_points(num_sensors, dim)
    sensors_k = sensors_0

    # iteration
    alpha = .0000001
    while check > 10**-8 and k < max_iter:
        sensors_k1 = sensors_k - alpha * grad(sensors_k)
        check = np.linalg.norm(sensors_k1 - sensors_k)
        sensors_k = sensors_k1
        if(k % 1000 == 0):
            print("NLLS k:" + str(k))
        k = k+1

    print("NLLS complete")
    return sensors_k
    
def grad(X):
    sensor_distance_sum = np.zeros((1,X.shape[1]))
    anchor_distance_sum = np.zeros((1,X.shape[1]))
    gradient = np.zeros(X.shape)

    for i, sensor_i in enumerate(X):
        for j, sensor_j in enumerate(X):
            if(i != j):
                sensor_distance_sum += (np.linalg.norm(sensor_i - sensor_j)**2 - \
                                        np.linalg.norm(sensors[i,:] - sensors[j,:])**2) * \
                                       (sensor_i - sensor_j)

        for k, anchor_k in enumerate(anchors):
            anchor_distance_sum += (np.linalg.norm(anchor_k - sensor_i)**2 - \
                                    np.linalg.norm(anchors[k,:] - sensors[i,:])**2) * \
                                   (sensor_i - anchor_k)

        gradient[i,:] = 8*sensor_distance_sum + 4*anchor_distance_sum

    return gradient

In [12]:
def get_sensor_matches(sensors, results):
    matches = []
    for sensor, result in zip(sensors, results):
        matches.append(np.linalg.norm(sensor - result) < 10**(-6))
    return matches

In [9]:
#run simulation
dim = 2
num_sensors = 10
num_anchors = 4

anchors = generate_points(num_anchors, dim)
sensors = generate_points(num_sensors, dim)

SOCP_result = get_SOCP_result(sensors, anchors)
SDP_result = get_SDP_result(sensors, anchors)
NLLS_result = get_NLLS_matches(sensors, anchors)

SOCP complete
SDP complete
NLLS k:0
NLLS k:500
NLLS k:1000
NLLS k:1500
NLLS k:2000
NLLS k:2500
NLLS k:3000
NLLS k:3500
NLLS k:4000
NLLS k:4500
NLLS k:5000
NLLS k:5500
NLLS k:6000
NLLS k:6500
NLLS k:7000
NLLS k:7500
NLLS k:8000
NLLS k:8500
NLLS k:9000
NLLS k:9500


In [14]:
print(sensors)
print(get_sensor_matches(sensors,SOCP_result))
print(get_sensor_matches(sensors,SDP_result))
print(get_sensor_matches(sensors,NLLS_result))

[[ 1.97484834  1.69665244]
 [-1.97778303 -1.3966189 ]
 [-1.25784669 -1.69066805]
 [ 0.10999455 -1.72076001]
 [-0.57225127 -0.85888764]
 [-0.36876669 -0.03505139]
 [ 0.17546492  0.0899418 ]
 [ 1.48423434 -1.91628835]
 [-1.83964622  0.26857945]
 [ 0.5151761  -0.94136593]]
[False, False, False, False, True, True, True, False, False, False]
[True, True, True, True, True, True, True, True, True, True]
[False, False, False, False, False, False, False, False, False, False]
